## Tool Use

Tools are pre-defined functions that agents can use. Instead of writing arbitrary code, agents can call tools to perform actions, such as searching the web, performing calculations, reading files, or calling remote APIs. Because you can control what tools are available to an agent, you can control what actions an agent can perform.

note: Tool use is currently only available for LLMs that support OpenAI-comaptible tool call API.

In [ ]:
from autogen import config_list_from_json

myModelname="GPT35TUEBOFIRST" #your model Name

#Read config File AOAI_CONFIG_LIST.json
config_list = config_list_from_json(
    env_or_file="..\AOAI_CONFIG_LIST.json",
    filter_dict={"model": myModelname}
    )

# LLM config file for myModelname (your model Name)

llm_myModelName_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }

## 1. Creating Tools

Tools can be created as regular Python functions. For example, let’s create a calculator tool which can only perform a single operation at a time.

In [ ]:
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

2. Registering Tools

Once you have created a tool, you can register it with the agents that are involved in conversation.

TIP: Always provide a clear and concise description for the tool as it helps the agent's underlying LLM to understand the tool's usage.

In [ ]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done or fail.",
    llm_config=llm_myModelName_config,
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


#Function description
funDes="This is a simple calculator, recieve 2 arguments and an operator and return the result of the operation."

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="calculator", description=funDes)(calculator)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="calculator")(calculator)

#TIP Always provide a clear and concise description for the tool as it helps the agent's underlying LLM to understand the tool's usage.

## 3. Using Tool

Once the tool is registered, we can use it in conversation. In the code below, we ask the assistant to perform some arithmetic calculation using the calculator tool.

In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="What is 666 + 3333?")